# Basic Usage

This tutorial will go through the basic usage of tiltlib, such as how to initialize a sample, and plotting

In [ ]:
# Import the necessary packages
%matplotlib inline
from orix.vector import Miller, Vector3d
import numpy as np

from tiltlib import Axis, Sample

## Axis initialization
To rotate a sample, one needs rotation axes.
In tiltlib, these are represented as `Axis` objects.
These are initialized with a direction and a tilt range.

In [ ]:
x_tilt_axis = Axis(
    Vector3d.xvector(),
    min = -30,
    max = 30,
    degrees = True,
)
x_tilt_axis

## Sample initialization

The main workhorse of tiltlib is the `Sample`. 
This object consists of crystal orientations, and rotation axes.

This can be initialized in two ways: with a `CrystalMap` from Orix, or with `Orientation` and a `Phase` (also from Orix). 

In [ ]:
## Orientations

from orix.quaternion import Orientation
from orix.crystal_map import Phase

# Make a hexagonal crystal.
# The default lattice is cubic, so we edit it
phase = Phase(point_group="6/mmm")
phase.structure.lattice.setLatPar(gamma=120)

# Make some random data
oris = Orientation.random((10, 10))
oris.symmetry = phase.point_group
oris = oris.map_into_symmetry_reduced_zone()

# Initialize the sample
sample = Sample(oris, phase, [x_tilt_axis])
sample

In [ ]:
## CrystalMap

# Download a crystal map supplied by Orix
from orix.data import sdss_austenite
raw_xmap = sdss_austenite(allow_download=True)

# Initialize the sample object
sample = Sample.from_crystal_map(raw_xmap, [x_tilt_axis])

sample

## Cropping

`Sample`s can be cropped with Hyperspy ROIs

In [ ]:
# Create an IPF navigation signal
nav = sample.to_navigator()
nav.plot()

# Create ROIs
from hyperspy.roi import RectangularROI, CircleROI

rect = RectangularROI(left=81, top=24, right=96, bottom=32)
rect.interactive(nav, color="black")

circ = CircleROI(cx=47, cy=55, r=5.5, r_inner=0)

# For some reason, numpy throws an error with the circle
try:
    circ.interactive(nav, color="black")
except np.exceptions.DTypePromotionError:
    pass

In [ ]:
rect_sample = sample.crop(rect)
circ_sample = sample.crop(circ)

## Plotting

There are multiple plotting tools directly available from the `Sample` object.
Additionally, the `Orientation` member is always up-to-date with the rotation of the sample, making them available for custom plotting.

In [ ]:
## IPF colormap
sample.plot()

In [ ]:
## Orientations as IPF scatterplot
# Use one of the crops, as the full sample has so many different orientations
rect_sample.plot_orientations()

## Interactive plotting

Some plotting functionality of tiltlib is interactive, where you control the tilt axes with sliders

In [ ]:
# IPF colormaps
sample.plot_interactive()

In [ ]:
# IPF scatterplot
circ_sample.plot_orientations_interactive()